## 爬取Pokemon图片

In [7]:
import requests
from bs4 import BeautifulSoup
import re
import os
from IPython.display import Image
import time

# url 前缀
url_prefix = 'https://wiki.52poke.com'

# /wiki/宝可梦列表（按全国图鉴编号）/简单版

**1.爬取网页**

In [13]:
def crawl(url, url_suffix):
    '''
    根据传入的 url 获取html
    returns:
        html: 获取到的 html
    '''
    url = url + url_suffix
    html = requests.get(url).text
    return html

**2.解析网页**

In [17]:
def parse(html):
    '''
    根据传入的 html 找到我们需要的数据
    return:
        all_pokemon_urls: 所有 Pokemon 的url
    '''
    # 创建 html 实例
    soup = BeautifulSoup(html, 'html.parser')
    
    # 所以所有 href 标签内容
    all_hrefs = soup.find(class_='mw-content-ltr').find_all(class_='mw-redirect')
    
    # 提取所有链接
    all_urls = [url.get('href') for url in all_hrefs]
    
    # 仅提取格式为 `/wiki/Bulbasaur` 的所有链接
#     all_pokemon_urls = []
#     i = 0
#     while i < len(all_urls):
#         if i%2 != 0:
#             all_pokemon_urls.append(all_urls[i])
#             i += 1
    
#     return all_pokemon_urls
    return all_urls

In [15]:
# 获取 Pokemon html
html = crawl(url_prefix, '/wiki/宝可梦列表（按全国图鉴编号）/简单版')

In [18]:
# 获取所有 Pokemon 详细信息链接
all_pokemon_urls = parse(html)

In [19]:
all_pokemon_urls

['/wiki/%E3%83%95%E3%82%B7%E3%82%AE%E3%83%80%E3%83%8D',
 '/wiki/Bulbasaur',
 '/wiki/%E3%83%95%E3%82%B7%E3%82%AE%E3%82%BD%E3%82%A6',
 '/wiki/Ivysaur',
 '/wiki/%E3%83%95%E3%82%B7%E3%82%AE%E3%83%90%E3%83%8A',
 '/wiki/Venusaur',
 '/wiki/%E3%83%92%E3%83%88%E3%82%AB%E3%82%B2',
 '/wiki/Charmander',
 '/wiki/%E3%83%AA%E3%82%B6%E3%83%BC%E3%83%89',
 '/wiki/Charmeleon',
 '/wiki/%E3%83%AA%E3%82%B6%E3%83%BC%E3%83%89%E3%83%B3',
 '/wiki/Charizard',
 '/wiki/%E3%82%BC%E3%83%8B%E3%82%AC%E3%83%A1',
 '/wiki/Squirtle',
 '/wiki/%E3%82%AB%E3%83%A1%E3%83%BC%E3%83%AB',
 '/wiki/Wartortle',
 '/wiki/%E3%82%AB%E3%83%A1%E3%83%83%E3%82%AF%E3%82%B9',
 '/wiki/Blastoise',
 '/wiki/%E3%82%AD%E3%83%A3%E3%82%BF%E3%83%94%E3%83%BC',
 '/wiki/Caterpie',
 '/wiki/%E3%83%88%E3%83%A9%E3%83%B3%E3%82%BB%E3%83%AB',
 '/wiki/Metapod',
 '/wiki/%E3%83%90%E3%82%BF%E3%83%95%E3%83%AA%E3%83%BC',
 '/wiki/Butterfree',
 '/wiki/%E3%83%93%E3%83%BC%E3%83%89%E3%83%AB',
 '/wiki/Weedle',
 '/wiki/%E3%82%B3%E3%82%AF%E3%83%BC%E3%83%B3',
 '/wiki/Kakuna',


**根据 `first_level_links` 的链接，逐一爬取每个 Pokemon 的详细网页中的图片链接**

In [255]:
img_links = []
url_prefix = 'https://'

for i in range(10):
    soup3 = make_soup(first_level_links[i])
    l = soup3.find('table', class_='roundy bgwhite fulltable').find('img').get('data-srcset').split(',')[0]
    u = url_prefix + re.findall(r'media.*png', l)[0]
    img_links.append(u)
    
img_links

['https://media.52poke.com/wiki/thumb/2/21/001Bulbasaur.png/450px-001Bulbasaur.png',
 'https://media.52poke.com/wiki/thumb/7/73/002Ivysaur.png/450px-002Ivysaur.png',
 'https://media.52poke.com/wiki/thumb/a/ae/003Venusaur.png/450px-003Venusaur.png',
 'https://media.52poke.com/wiki/thumb/7/73/004Charmander.png/450px-004Charmander.png',
 'https://media.52poke.com/wiki/4/4a/005Charmeleon.png',
 'https://media.52poke.com/wiki/thumb/7/7e/006Charizard.png/450px-006Charizard.png',
 'https://media.52poke.com/wiki/thumb/3/39/007Squirtle.png/450px-007Squirtle.png',
 'https://media.52poke.com/wiki/0/0c/008Wartortle.png',
 'https://media.52poke.com/wiki/thumb/0/02/009Blastoise.png/450px-009Blastoise.png',
 'https://media.52poke.com/wiki/5/5d/010Caterpie.png']

In [252]:
# 创建文件夹路径
path = './pokemon-img/'
if not os.path.exists(path):
    os.makedirs(path)
else:
    print('文件路径已存在！')

文件路径已存在！


In [257]:
file_name = img_links[0].split('/')[-1]
file_name

'450px-001Bulbasaur.png'

In [259]:
def download_img(path, img_link, index):
    '''
    根据 img_link 下载 Pokemon 图片
    
    params:
        path: 文件保存的文件夹路径
        img_link: 需下载的图片链接
        index: 用于遍历下载链接的坐标
    '''
    r = requests.get(img_link, stream=True)
    
    # 保存的文件名
    file_name = img_links[index].split('/')[-1]
    
    # 文件路径
    file_path = path+file_name
    
    # 需指定文件名，否则会出现 [Errno 21] Is a directory 错误
    with open(file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=32):
            f.write(chunk)
        print('下载 {} 成功！'.format(file_name))

In [261]:
for i in range(10):
    download_img(path, img_links[i], i)

下载 450px-001Bulbasaur.png 成功！
下载 450px-002Ivysaur.png 成功！
下载 450px-003Venusaur.png 成功！
下载 450px-004Charmander.png 成功！
下载 005Charmeleon.png 成功！
下载 450px-006Charizard.png 成功！
下载 450px-007Squirtle.png 成功！
下载 008Wartortle.png 成功！
下载 450px-009Blastoise.png 成功！
下载 010Caterpie.png 成功！
